In [28]:
# -*- coding: utf-8 -*-
from __future__ import unicode_literals

import numpy as np
from keras.models import load_model
import sys
sys.path.insert(0, '../../../')
from prepare_data import *

from keras import optimizers
from keras.models import Model
from keras.layers import Dense, Dropout, Input
from keras.layers.merge import concatenate
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers import Flatten
from keras.models import load_model
# from keras import backend as Input
np.random.seed(7)
import pickle

In [30]:
data = Data('l', shuffle_all_inputs=False)
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
slo_feature_dictionary = data._create_slovene_feature_dictionary()
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)
accented_vowels = ['ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']

environment = {}
environment['dictionary'] = dictionary
environment['max_word'] = max_word
environment['max_num_vowels'] = max_num_vowels
environment['vowels'] = vowels
environment['accented_vowels'] = accented_vowels
environment['slo_feature_dictionary'] = slo_feature_dictionary
environment['feature_dictionary'] = feature_dictionary
environment['syllable_dictionary'] = syllable_dictionary
output = open('../../../environment.pkl', 'wb')
pickle.dump(environment, output)
output.close()

In [15]:
data = Data('l', shuffle_all_inputs=False)

In [16]:
letter_location_model, syllable_location_model, syllabled_letters_location_model = data.load_location_models(
    '../cnn_dictionary/v3_10/20_test_epoch.h5',
    '../syllables/v2_4/20_test_epoch.h5',
    '../syllabled_letters/v2_5_3/20_test_epoch.h5')

In [17]:
letter_type_model, syllable_type_model, syllabled_letter_type_model = data.load_type_models(
    '../../accent_classification/letters/v2_1/20_test_epoch.h5',
    '../../accent_classification/syllables/v1_0/20_test_epoch.h5',
    '../../accent_classification/syllabled_letters/v1_0/20_test_epoch.h5')

In [18]:
# generates text with every word in new line
with open('../../../../reldi-tagger/test_original') as f:
    original_text = f.readlines()
original_text = ''.join(original_text)
# print(original_text)
text_with_whitespaces = original_text.replace(',', ' ,').replace('.', ' .').replace('\n', ' ').replace("\"", " \" ").replace(":", " :").replace("ć", "č").replace('–', '-')
# print('-------------------------------------------------')
text_with_whitespaces = '\n'.join(text_with_whitespaces.split())
text_with_whitespaces += '\n\n'
# print(text_with_whitespaces)
with open('../../../../reldi-tagger/test_with_whitespaces', "w") as text_file:
    text_file.write(text_with_whitespaces)
    
# generates text with PoS tags
import subprocess

myinput = open('../../../../reldi-tagger/test_with_whitespaces', 'r')
myoutput = open('../../../../reldi-tagger/test_result', 'w')
# print(myinput.readlines())
python3_command = "../../../../reldi-tagger/tagger.py sl"  # launch your python2 script using bash

process = subprocess.run(python3_command.split(), stdin=myinput, stdout=myoutput)

# generates interesting words
pointless_words = ['.',',','\"',':', '-']
with open('../../../../reldi-tagger/test_result', "r") as text_file:
    tagged_input_words = []
    for x in text_file.readlines()[:-1]:
        splited_line = x[:-1].split('\t')
        if splited_line[0] not in pointless_words and not any(char.isdigit() for char in splited_line[0]):
            tagged_input_words.append([splited_line[0].lower(), '', splited_line[1], splited_line[0].lower()])
#     tagged_input_words = [x[:-1].split('\t') for x in text_file.readlines() if x[:-1].split('\t')[0] not in pointless_words][:-1]

In [13]:
tagged_input_words

[['izbruhi', '', 'Ncmpn', 'izbruhi'],
 ['na', '', 'Sl', 'na'],
 ['soncu', '', 'Ncnsl', 'soncu'],
 ['so', '', 'Va-r3p-n', 'so'],
 ['že', '', 'Q', 'že'],
 ['večkrat', '', 'Rgc', 'večkrat'],
 ['pokazali', '', 'Vmep-pm', 'pokazali'],
 ['zobe', '', 'Ncmpa', 'zobe'],
 ['našim', '', 'Ps1mpdp', 'našim'],
 ['satelitom', '', 'Ncmpd', 'satelitom'],
 ['posledično', '', 'Rgp', 'posledično'],
 ['našim', '', 'Ps1mpdp', 'našim'],
 ['mobilnim', '', 'Agpmpd', 'mobilnim'],
 ['telefonom', '', 'Ncmpd', 'telefonom'],
 ['navigaciji', '', 'Ncfsl', 'navigaciji'],
 ['celo', '', 'Q', 'celo'],
 ['električnemu', '', 'Agpnsd', 'električnemu'],
 ['omrežju', '', 'Ncnsd', 'omrežju'],
 ['a', '', 'Ncmsn', 'a'],
 ['vesoljskega', '', 'Agpnsg', 'vesoljskega'],
 ['vremena', '', 'Ncnsg', 'vremena'],
 ['še', '', 'Q', 'še'],
 ['ne', '', 'Q', 'ne'],
 ['moremo', '', 'Vmpr1p', 'moremo'],
 ['napovedati', '', 'Vmen', 'napovedati'],
 ['kako', '', 'Rgp', 'kako'],
 ['bi', '', 'Va-c', 'bi'],
 ['ga', '', 'Pp3msa--y', 'ga'],
 ['lahko', '

In [6]:
content = data._read_content('../../../data/SlovarIJS_BESEDE_utf8.lex')
dictionary, max_word, max_num_vowels, vowels, accented_vowels = data._create_dict(content)
feature_dictionary = data._create_feature_dictionary()
syllable_dictionary = data._create_syllables_dictionary(content, vowels)

In [24]:
accented_vowels = ['ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']

In [25]:
# X, X_other_features, fake_y = generate_X_and_y(dictionary, max_word, max_num_vowels, tagged_input_words, vowels, accetuated_vowels, feature_dictionary, 'who cares', shuffle=False)

#pos = 45
%run ../../../prepare_data.py
data = Data('l', shuffle_all_inputs=False)
predictions = data.get_ensemble_location_predictions(tagged_input_words, letter_location_model, syllable_location_model, syllabled_letters_location_model,
                                          dictionary, max_word, max_num_vowels, vowels, accented_vowels, feature_dictionary, syllable_dictionary)

#print(predictions)
if 'A' not in vowels:
    vowels.extend(['A', 'E', 'I', 'O', 'U'])
accented_words = [data.assign_location_stress(tagged_input_words[i][0][::-1], data.decode_y(predictions[i]), vowels)[::-1] for i in range(len(tagged_input_words))]

#print(accented_words[:20])
#print(tagged_input_words[:20])

words_and_accetuation_loc = [[tagged_input_words[i][0], data.decode_y(predictions[i])] for i in range(len(tagged_input_words))]

original_text_list = list(original_text)
original_text_lowercase = original_text.lower()
end_pos = 0
for word in words_and_accetuation_loc:
    posit = original_text_lowercase.find(word[0], end_pos)
    if posit != -1:
        start_pos = posit
        end_pos = start_pos + len(word[0])
    
    original_text_list[start_pos:end_pos] = list(data.assign_location_stress(''.join(original_text_list[start_pos:end_pos][::-1]), word[1], vowels)[::-1])

    
    #print(start_pos)
#    if 4005 < start_pos < 4028:
#        print(word[0])
#        print(original_text_list[start_pos:end_pos])
#        print(start_pos)
    
#    if word[0] == "zalivu":
#        print("TEST")
#        print(start_pos)
#        print(end_pos)
#        print(original_text_list[start_pos:end_pos])
#        print(''.join(original_text_list))

print(''.join(original_text_list))

/////////////////////////////////////////////////////////////////////////////////////////
[[  2.10678263e-04   9.93271589e-01   5.18704765e-04   1.14275721e-07
    2.54067871e-13   2.64285016e-17   2.18240356e-21   6.20236917e-24
    1.42706811e-21   1.41564975e-28]
 [  3.89118418e-02   8.47151801e-02   3.24135799e-05   4.05992717e-15
    3.09737105e-16   8.77186102e-11   1.06644402e-10   1.23290537e-15
    3.55467122e-08   5.67882674e-21]
 [  9.34812135e-07   9.99476612e-01   6.02825168e-08   3.19877625e-17
    2.81368350e-26   5.04405889e-35   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  9.88366842e-01   7.11048953e-03   5.13283149e-09   1.44280518e-10
    6.99923192e-12   3.75810223e-14   7.49483043e-15   6.17138854e-14
    4.33775195e-14   4.44727257e-29]
 [  8.99618804e-01   7.83890709e-02   7.81788640e-06   1.86424265e-10
    1.74876356e-14   3.45468338e-15   1.33400046e-13   1.60374730e-16
    7.48545215e-13   1.26708724e-23]
 [  3.05425376e-01   6.90

/////////////////////////////////////////////////////////////////////////////////////////
[[  1.30707131e-13   1.00000000e+00   3.27903191e-14   1.71698103e-26
    2.67368097e-29   9.97666048e-35   0.00000000e+00   1.61916024e-36
    0.00000000e+00   0.00000000e+00]
 [  1.23546300e-02   1.33980706e-04   8.06673006e-10   1.68207267e-12
    1.92988227e-06   2.42033821e-07   1.35941855e-10   8.14914785e-08
    3.18686966e-10   7.50316066e-21]
 [  1.51177477e-15   1.00000000e+00   1.88466235e-12   2.00318617e-25
    2.36180250e-29   1.48853327e-33   0.00000000e+00   0.00000000e+00
    0.00000000e+00   0.00000000e+00]
 [  9.84390974e-01   2.90597363e-05   2.75492712e-05   7.00570324e-10
    1.89383134e-10   1.49957251e-11   1.12752863e-10   3.53997837e-14
    1.31105917e-16   4.92311999e-19]
 [  9.87746596e-01   5.93861984e-03   6.00138737e-05   1.22034714e-06
    3.04159443e-07   1.41836418e-08   2.93405300e-09   2.55820898e-09
    1.26385946e-09   4.43041600e-14]
 [  1.69836164e-06   9.99

/////////////////////////////////////////////////////////////////////////////////////////
[[  7.68751551e-09   9.99985874e-01   2.14351689e-06   1.88919946e-17
    9.22816585e-21   1.22504893e-21   4.98980061e-36   6.06510315e-37
    2.43257132e-33   0.00000000e+00]
 [  1.04926410e-03   2.49189558e-03   2.29439756e-09   9.64960330e-14
    1.31074373e-07   1.87512671e-06   3.54630062e-12   5.16962935e-08
    1.03037141e-10   1.53020901e-21]
 [  3.24184803e-04   9.98982787e-01   1.50819929e-07   6.03612368e-11
    4.54062612e-12   1.01809762e-13   7.08547322e-21   9.98586531e-22
    1.12581117e-22   1.24080517e-35]
 [  9.48761642e-01   3.23764645e-02   2.39664256e-12   8.54690599e-14
    3.34223405e-09   1.63910945e-13   4.75882761e-15   1.20896716e-14
    1.29485194e-22   1.27796214e-36]
 [  9.27000225e-01   5.92121892e-02   7.07522618e-09   2.06750155e-12
    4.71528372e-09   9.74130887e-10   1.64832763e-12   1.33748134e-12
    3.26861577e-18   1.53783010e-26]
 [  2.59232387e-04   9.99

In [ ]:
Izbrúhi na sóncu só žé véčkrat pokazáli zóbe nášim satelítom, poslédično nášim mobílnim telefónom, navigáciji, célo eléktričnemu omréžju. Á vesóljskega vreména šé né móremo napovédati – kakó bí ga láhko, se tá téden na Blédu pogovárja okóli 70 znánstvenikov Evrópske vesóljske agéncije, ki jé sebój pripeljála svójo nájvéčjo ikóno, británca Mátta Taylorja.

In [10]:
#%run ../../../prepare_data.py
accented_vowels = ['ŕ', 'á', 'ä', 'é', 'ë', 'ě', 'í', 'î', 'ó', 'ô', 'ö', 'ú', 'ü']
#pos = 87
#print(predictions[3])
#print(np.around(predictions)[3])


location_y = np.around(predictions)
type_predictions = data.get_ensemble_type_predictions(tagged_input_words, location_y, letter_type_model, syllable_type_model, syllabled_letter_type_model,
                                          dictionary, max_word, max_num_vowels, vowels, accented_vowels, feature_dictionary, syllable_dictionary)


input_words = [el[0] for el in tagged_input_words]
words = data.assign_stress_types(type_predictions, input_words, location_y, vowels, accented_vowels)

#print(original_text)

original_text_list = list(original_text)
original_text_lowercase = original_text.lower()
end_pos = 0
for i in range(len(words)):
    posit = original_text_lowercase.find(input_words[i], end_pos)
    if posit != -1:
        start_pos = posit
        end_pos = start_pos + len(words[i])
        
        orig_word = original_text_list[start_pos:end_pos]
        new_word = list(words[i])
        for j in range(len(orig_word)):
            if orig_word[j].isupper():
                new_word[j] = new_word[j].upper()
        
        original_text_list[start_pos:end_pos] = new_word

print(''.join(original_text_list))

Izbrúhi na sóncu só žé vëčkrat pokazáli zóbe nášim satelítom, poslédično nášim mobílnim telefónom, navigáciji, célo eléktričnemu omréžju. Á vesóljskega vreména šë në móremo napovédati – kakó bî ga láhko, se tá téden na Blédu pogovárja okóli 70 znánstvenikov Evrópske vesóljske agéncije, ki jé sebój pripeljála svôjo näjvéčjo ikóno, británca Mátta Taylorja.



In [65]:
%run ../../../prepare_data.py
data = Data('l', shuffle_all_inputs=False)
location_accented_words, accented_words = data.accentuate_word(tagged_input_words, letter_location_model, syllable_location_model, syllabled_letters_location_model,
                        letter_type_model, syllable_type_model, syllabled_letter_type_model,
                        dictionary, max_word, max_num_vowels, vowels, accented_vowels, feature_dictionary, syllable_dictionary)

In [66]:
print(tagged_input_words[:20])
print(location_accented_words[:40])
print(accented_words[:40])

[['uradni', '', 'Agpmpn', 'uradni'], ['podatki', '', 'Ncmpn', 'podatki'], ['policije', '', 'Ncfsg', 'policije'], ['kažejo', '', 'Vmpr3p', 'kažejo'], ['na', '', 'Sa', 'na'], ['precej', '', 'Rgp', 'precej'], ['napete', '', 'Appfpa', 'napete'], ['razmere', '', 'Ncfpa', 'razmere'], ['v', '', 'Sl', 'v'], ['piranskem', '', 'Agpmsl', 'piranskem'], ['zalivu', '', 'Ncmsl', 'zalivu'], ['je', '', 'Va-r3s-n', 'je'], ['danes', '', 'Rgp', 'danes'], ['poročala', '', 'Vmpp-sf', 'poročala'], ['oddaja', '', 'Ncfsn', 'oddaja'], ['do', '', 'Sg', 'do'], ['danes', '', 'Rgp', 'danes'], ['se', '', 'Px------y', 'se'], ['je', '', 'Va-r3s-n', 'je'], ['zgodilo', '', 'Vmep-sn', 'zgodilo']]
['urádni', 'podátki', 'policíje', 'kážejo', 'na', 'precéj', 'napéte', 'razmére', 'v', 'piránskem', 'zalívu', 'jé', 'danés', 'poróčala', 'oddája', 'do', 'danés', 'se', 'jé', 'zgodílo', 'žé', 'incidéntov', 'v', 'katérih', 'só', 'hrváška', 'plovíla', 'krát', 'zaplúla', 'v', 'vóde', 'ki', 'veljájo', 'kot', 'obmóčje', 'slovénskega', 